# Trabajo Práctico N° 5
<br/>

## Curso de Especialización en Inteligencia Artificial
## Visión por computadora 1
<br/>

### Autor: Maximiliano Torti
### Fecha: 05/06/21
<br/>

#### Enunciado:<br/>
Utilizando la webcam
1. Implementar un algoritmo de seguimiento con CamShift que <br/>
 a. Permita elegir la ROI del objeto a seguir <br/>
 b. Permita cambiar la escala de la ventana (no necesariamente la orientación) <br/>


In [24]:
# Importamos las librerías necesarias
# %matplotlib inline
%matplotlib
import time
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt

Using matplotlib backend: Qt5Agg


In [25]:
# Clase que permite seleccionar el ROI
class RoiSelectionClass:
    
    x1 = None
    y1 = None
    x2 = None
    y2 = None
    drawing = False
    new = False
    
    def __init__(self):
        pass
    
    # Callback eventos de mouse
    def mouse_event(self, event,x,y,flags,param):
        if event == cv.EVENT_LBUTTONDOWN:
            # Al presionar y mantener el click, inicia evento de dibujo del ROI
            self.new = False
            self.x1 = x
            self.x2 = x
            self.y1 = y
            self.y2= y
            self.drawing = True
        elif event == cv.EVENT_MOUSEMOVE:
            # Al movel el mouse con el boton presionado dibuja el ROI
            if self.drawing:
                self.x2 = x
                self.y2 = y
        elif event == cv.EVENT_LBUTTONUP:
            # Al soltar el mouse actualiza nuevo ROI valido
            self.drawing = False
            self.x2 = x
            self.y2 = y
            if self.x2 > (self.x1 + 10) and self.y2 > (self.y1 + 10) :
                self.new = True
            else:
                self.new = False
                
    def is_new(self):
        # Flag nuevo ROI
        if self.new:
            self.new = False
            return True
        else:
            return False
    
    def is_drawing(self):
        # ROI dibujandose
        return self.drawing
    
    def get_roi(self):
        # Obtener cordenadas de ROI
        return self.x1 , self.y1, self.x2, self.y2
    

In [27]:
# Criterio terminación para Camshift
term_crit = ( cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 1, 10 )

# Obtengo stream de datos de la webcam
cap = cv.VideoCapture(0)
# Chequeo que la camara este disponible
if not cap.isOpened():
    raise IOError("WebCam no disponible")

# Creo clase de seleccion de ROI
roiSelect = RoiSelectionClass()

# Abro ventana y asigno callback eventos de mouse
cv.namedWindow('Input')
cv.setMouseCallback('Input',roiSelect.mouse_event)

# Variable para comenzar y frenar el algoritmo de tracking ()
run_tracking= False
fixed_roi = False

while True:
    # Leo frame de la webcam
    ret, frame_orig = cap.read()
    
    if ret == True:
        
        # Elimino zonas negras
        frame_orig = frame_orig[60:420,:]
        
        # Frame para aplicar dibujos
        frame = frame_orig.copy()
        
        if roiSelect.is_drawing():
            # ROI dibujandose, freno el tracking y dibujo ventana en azul
            run_tracking=False
            x_temp1, y_temp1, x_temp2, y_temp2 = roiSelect.get_roi()
            frame = cv.rectangle(frame, (x_temp1,y_temp1), (x_temp2,y_temp2), [255,0,0], 2)

        if roiSelect.is_new():
            # Nuevo ROI, extraigo características e inicio tracking
            run_tracking=True
            x1, y1, x2, y2 = roiSelect.get_roi()
            track_window = (x1, y1, x2-x1, y2-y1)
            roi = frame_orig[y1:y2, x1:x2]
            # Paso a hsv
            hsv_roi =  cv.cvtColor(roi, cv.COLOR_BGR2HSV)
            # Máscara para el histograma
            mask = cv.inRange(hsv_roi, np.array((0.,60.,32.)), np.array((180.,255.,255.)))
            # Histograma con máscara
            roi_hist = cv.calcHist([hsv_roi],[0],mask,[180],[0,180])
            # Histograma normalizado
            cv.normalize(roi_hist,roi_hist,0,255,cv.NORM_MINMAX)

        if run_tracking:
            # Nuevo frame a hsv
            hsv = cv.cvtColor(frame_orig, cv.COLOR_BGR2HSV)
            dst = cv.calcBackProject([hsv],[0],roi_hist,[0,180],1)
            # Aplicamos camshift al nuevo frame
            ret, track_window_temp = cv.CamShift(dst, track_window, term_crit)
            # Podemos elegir entre ventana de escala fija o escala variable según CamShift
            if fixed_roi:
                x,y = track_window_temp[0:2]
                w,h = track_window[2:4]
            else :
                x,y,w,h = track_window_temp
            # Dibujamos la ventana sobre la imagen
            track_window = (x,y,w,h)
            frame = cv.rectangle(frame, (x,y), (x+w,y+h), [0,0,255],2)
        
        cv.imshow('Input',frame)
    
    c = cv.waitKey(1)
    
    # Con las teclas + y - se cambia a ventana fija y se agrandan o achican
    # Con tecla / se restaura a escala por CamShift
    if c == 43:
        fixed_roi = True
        x,y,w,h = track_window
        track_window = (x,y,round(w*1.1),round(h*1.1))
    
    if c == 45:
        fixed_roi = True
        x,y,w,h = track_window
        track_window = (x,y,round(w*0.9),round(h*0.9))
        
    if c == 47:
        fixed_roi = False
    
    # Finalizar con escape 
    if c == 27:
        break

cap.release()
cv.destroyAllWindows()